In [1]:
import tarfile,sys
import pandas as pd
import os
import re as regex
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import seaborn as sns
import numpy
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import chakin

## Read from CSV file containing features

In [2]:
#testdata = pd.read_csv('test.csv')
traindata = pd.read_csv('train-small')
traindata.head()

,Unnamed: 0,class,user_name,id,tweet,avgsentiment,favorite_count,firstperson,geo,possibly_sensitive,retweet_count,retweeted,tsugawa,number_of_uppercase,number_of_exclamation,number_of_question,number_of_ellipsis
0,0,depression,k1Tgty92JWM,6195046850825297023,"RT @FpJgkXU6: My cat is sad because, having be...",0.112638,0,2670,NaN,0.0,249,False,0,2,0,0,0
1,0,control,f8Da_Rvi,8522289883066978092,Running Boston for Children's Hospital again. ...,0.118159,0,142,NaN,0.0,1,False,0,0,0,0,0
2,0,depression,hehQgSVVRpv,322688550056773303,My mom is just swell like that. :)####In other...,0.047492,0,2562,NaN,NaN,0,False,0,1,0,0,0
3,0,control,qPQRyK39gnG,2184602040112857282,Ain't nothing like starting your morning with ...,0.099925,0,2592,NaN,NaN,0,False,0,1,1,0,0
4,0,depression,j_1H5RgR2_6,2834216959061162834,RT @GAndbcDpitkbVU: #ThePinkPrintCoverTomorrow...,0.211423,0,2891,NaN,NaN,10,False,0,2,0,0,0


In [3]:
# b = pd.DataFrame(traindata['tweet'].str.split('####').tolist(), index=[traindata['user_name'], traindata['class'], traindata['avgsentiment'], traindata['favorite_count'], traindata['firstperson'], traindata['geo'], traindata['possibly_sensitive'], traindata['retweet_count'], traindata['retweeted'], traindata['tsugawa'], traindata['number_of_uppercase'], traindata['number_of_exclamation'], traindata['number_of_question'], traindata['number_of_ellipsis']]).stack()
# b = b.reset_index()[[0, 'user_name','class', 'avgsentiment', 'favorite_count', 'firstperson', 'geo', 'possibly_sensitive', 'retweet_count', 'retweeted', 'tsugawa', 'number_of_uppercase', 'number_of_exclamation', 'number_of_question', 'number_of_ellipsis']] 
# b.columns = ['tweet', 'user_name','class', 'avgsentiment', 'favorite_count', 'firstperson', 'geo', 'possibly_sensitive', 'retweet_count', 'retweeted', 'tsugawa', 'number_of_uppercase', 'number_of_exclamation', 'number_of_question', 'number_of_ellipsis'] 
# final_train_data = b
b = pd.DataFrame(traindata['tweet'].str.split('####').tolist(), index=[traindata['user_name'], traindata['class']]).stack()
b = b.reset_index()[[0, 'user_name','class']] # var1 variable is currently labeled 0
b.columns = ['tweet', 'user_name','class'] # renaming var1
final_train_data = b

In [4]:
final_train_data.head()

,tweet,user_name,class
0,"RT @FpJgkXU6: My cat is sad because, having be...",k1Tgty92JWM,depression
1,@r2kbk0ooSf7 @gJX64gO487acDT hope it works out...,k1Tgty92JWM,depression
2,@r2kbk0ooSf7 Good thanks. Really good. Enjoyin...,k1Tgty92JWM,depression
3,Kids have a new game. Let's try to strangle ea...,k1Tgty92JWM,depression
4,4yo thinks she has one foot bigger than the ot...,k1Tgty92JWM,depression


## Clean Up data

In [5]:
def remove_by_regex(tweets, regexp):
    tweets.loc[:, "tweet"].replace(regexp, "", inplace=True)
    return tweets

def remove_urls(tweets):
    return remove_by_regex(tweets, regex.compile(r"http.?://[^\s]+[\s]?"))

def remove_na(tweets):
    return tweets[tweets["tweet"] != "Not Available"]

def remove_special_chars(tweets):  # it unrolls the hashtags to normal words
    for remove in map(lambda r: regex.compile(regex.escape(r)), [",", ":", "\"", "=", "&", ";", "%", "$",
                                                                 "@", "%", "^", "*", "(", ")", "{", "}",
                                                                 "[", "]", "|", "/", "\\", ">", "<", "-",
                                                                 "!", "?", ".", "'",
                                                                 "--", "---", "#"]):
        tweets.loc[:, "tweet"].replace(remove, "", inplace=True)
    return tweets

def remove_usernames(tweets):
    return remove_by_regex(tweets, regex.compile(r"@[^\s]+[\s]?"))

def remove_numbers(tweets):
    return remove_by_regex(tweets, regex.compile(r"\s?[0-9]+\.?[0-9]*"))

In [6]:
data = final_train_data
data = remove_urls(data)
data = remove_na(data)
data = remove_usernames(data)
data = remove_special_chars(data)
data = remove_numbers(data)
#data.cleanup(TwitterCleanuper())

In [7]:
data['tweet'].to_csv('tweets.csv')

In [8]:
def tokenize(tweet):
        # Tokenize file content
    ids = torch.ByteTensor(len(tweet.encode()))
    token = 0
    for char in tweet.encode():
        ids[token] = char
        token += 1

    return ids

def batchify(data, bsz):
    nbatch = data.size(0) // bsz
    data = data.narrow(0, 0, nbatch * bsz)
    data = data.view(bsz, -1).t().contiguous()
    return data   

In [9]:
x = data.query('tweet != ""')
x.shape
data=x
data = data[data.tweet.str.split(' ').str.len() > 1]
data=data.reset_index(drop=True)

In [10]:
data.shape

(86415, 3)

In [11]:
def copy_state(state):
    if isinstance(state, tuple):
    	return (Variable(state[0].data), Variable(state[1].data))
    else:
    	return Variable(state.data) 

def batchify(data, bsz):
    nbatch = data.size(0) // bsz
    data = data.narrow(0, 0, nbatch * bsz)
    data = data.view(bsz, -1).t().contiguous()
    return data        


In [15]:
def update_lr(optimizer, lr):
    for group in optimizer.param_groups:
        group['lr'] = lr
    return
	
def clip_gradient_coeff(model, clip):
    """Computes a gradient clipping coefficient based on gradient norm."""
    totalnorm = 0
    for p in model.parameters():
        modulenorm = p.grad.data.norm()
        totalnorm += modulenorm ** 2
    totalnorm = math.sqrt(totalnorm)
    return min(1, clip / (totalnorm + 1e-6))

def calc_grad_norm(model):
    """Computes a gradient clipping coefficient based on gradient norm."""
    totalnorm = 0
    for p in model.parameters():
        modulenorm = p.grad.data.norm()
        totalnorm += modulenorm ** 2
    return math.sqrt(totalnorm)
    
def calc_grad_norms(model):
    """Computes a gradient clipping coefficient based on gradient norm."""
    norms = []
    for p in model.parameters():
        modulenorm = p.grad.data.norm()
        norms += [modulenorm]
    return norms
    
def clip_gradient(model, clip):
    """Clip the gradient."""
    totalnorm = 0
    for p in model.parameters():
        print(p)
        p.grad.data = p.grad.data.clamp(-clip,clip)

In [17]:

class Net(nn.Module):
    def __init__(self, features, cls_size):
        super(Net, self).__init__()
        self.hidden_size = 256
        self.embedding = nn.Embedding(features, self.hidden_size)
        self.rnn1 = nn.GRU(input_size=features,
                            hidden_size=self.hidden_size)
        self.dense1 = nn.Linear(self.hidden_size, cls_size)

    def forward(self, x, hidden):
        output, hidden = self.rnn1(x, hidden)
        #output = self.dense1(output)
        return output, hidden

    def init_hidden(self, batch_size=1):
        weight = next(self.parameters()).data
        return Variable(weight.new(1, batch_size, self.hidden_size).zero_())

def var(x):
    x = Variable(x)
    return x

embed = nn.Embedding(256, 50)    
embed_optimizer = optim.SGD(embed.parameters(), lr=0.005)

model = Net(features=50, cls_size=256)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
hiddens=[]
rnn = model

def train():
    hiddenCounter=0
    model.train()
    batch_size = 1
    hidden_init = model.init_hidden(1)
    loss_avg = 0
    for s in range(data.shape[0]):
        print()
        model.zero_grad()
        embed_optimizer.zero_grad()
        hidden = hidden_init
        loss =0
        tweet=data.loc[s].tweet
        print (tweet)
        emb = tokenize(tweet)
        embLen=len(emb)
        for i in range(embLen): 
            character1=Variable((torch.Tensor([emb[i]])).long())
            character1=embed(character1)
            character1=character1.view(1,1,50)

            output, hidden = model(character1, var(hidden.data))
            output=output.view(1,256)

            if i+1 < embLen: 
                character2=Variable((torch.Tensor([emb[i+1]])).long())
                loss += criterion(output, character2)
        
        loss.backward()
        tempHidden=hidden.view(256)
        tempHidden=list(tempHidden.data.numpy())
        hiddens.append(tempHidden)
        hidden_init = copy_state(hidden)
        optimizer.step()
        embed_optimizer.step()
        loss_avg = .99*loss_avg + .01*loss.data[0]/embLen

        if s % 10000 == 0:
            print(epoch, s, loss_avg)

for epoch in range(1):
    print(model.parameters())
    hiddens=[]
    train()

<generator object Module.parameters at 0x7ff6634abb48>

RT My cat is sad because having been targeted by Internet trolls he must now live with a hour bodyguard 
0 0 0.05482186537522536

hope it works out well Xx

Good thanks Really good Enjoying life on the other side How are you

Kids have a new game Lets try to strangle each other Brilliant This cant possibly go wrong

yo thinks she has one foot bigger than the other and is utterly fascinated by this fact Shes making a bid for new shoe

What are the odds of theyo remembering its hot today and drinking lots of water zilch I might save time by recording my voice

lol something happens once and people just dont forget

Im really good Things are going well How are you Xxx

Im sorry It gets complicated managing work Twitter accounts and Im always paranoid Ill tweet from the wrong one

RT You can only genuinely like yourself if you can acknowledge that you have or are a work in progress continually developi…

Those arent really my areas bu


If is as shit as New Years Eve is turning out to be I might not bother with it

We skyped the abroad relies too It does help Had a lovely time thanks Hope you have Xx

I have a hangover and theyo hasnt stopped shouting singing and talking since she got up Where is the quiet button

Nothing says social pariah quite like a veruca sock

Ahyos at schoolyos at preschool I have tea cake and netflix and Im not moving till I have to pick theyo up

Could it actually be bedtime and weve all got confused hopeful face

Is it really morning or is someone playing a sick joke Ive had next to no sleep and my phones playing up again grumps off

They really are sometimes

Oh that sounds good 

He wouldnt hear He sleeps too heavily sighs

Lets hope so X

Things have improved Till the next phase Poor little thing Teething is just rubbish

Seriously Cant they just be a bit careful He use to fall on the bed putting his socks on and wake me up shaking the bed

my kids have gone through various early morning


My children have watched no TV today because something arrived in a big box So the entire evening has been spent in the box

Suddenly it sounds fabulous

Court on your day off is just mean

I like to live in hope

Im so tired Id give away chocolate for more sleep This may shock people

Mmmm contemplates a very big decision there I think it depends how quickly and sensibly they get dressed 

Morning peeps Cant think of many things I wouldnt give away now for another hours in bed

Lol I like your thinking

With the Just because or I just did reasoning Kids Whod have them if they knew

Nothing like starting the day with a large row with ayo Is it bedtime yet

Much sympathy Weeks like that are tough

Its really getting on my nerves How are you

Lol I know I few people whove had a cold its taken forever to throw off Hoping thats not happening to me

Im hoping to feel like that when my germs finally take a hike

Eugh Monday And I still have THE COLD Hows everyone else doing

RT My letter to


Hope youre OK Cars can be mendedreplaced Harder for people Xx

Welcome to my world

That sounds fantastic Were having a chilled out couple of days away but otherwise not much planned Enjoy

Me tooyo is beyond shattered

yo does this If it comes out full IME theyve drunk none of it If they fill it up again theyre thirsty + drink some

I know Who doesnt have normal sugar but has the others What can I say I like baking

End of term v possibly Or is he drinking water in school Mine doesnt amp this is an ongoing problem Affects lots of things

Ive just had to put icing sugar in the plumbers tea as weve run out of normal It was either that or vanilla sugar or light muscovado

Ha ha Might have to invest in a canoe

YES Cant wait for a break  Off for a couple of days away tomorrow all together Will be lovely if wet

RT Vitamin D tablets for free Why bother when theres an app for that sun

Im going to try to remember the moment when theyre trying to kill each other

This morning I heardyo wake


Yo has decided she either wants to be an Angelina or a mermaid or a fairy or an astronaut random

And one does have to live

Never a truer word spoken You want to develop yourself but not bankrupt yourself

Here here wellsaidthatlady

Sounds promising  Let me know if you have any joy

If anyone happens to know this Saturdays lotto numbers Id love you forevs if you share

No definitely not More than anything I could climb into bed right now

Having a very tired week Can I play a substitute and go back to bed

It is a genius strategy But disappointing when youre winning

Half way through snakes and ladders and it looks like victory is minesoyo announces its boring and says lets play something else

Why do coat their raisins in oil Its unnecesary Plus palm oil is not a good choice Id like my raisins wo oil + deforestation

I forgot to wash my hair last night plasters it in dry shampoo

I know Its not right

Lol always nice to be missed  I have been a busy little bee But expect normal ser

...


RT The Hits SuperBass   EMABiggestFansNickiMinaj 

RT Me wonder why the Barbz arent voting EMABiggestFansNickiMinaj RT  vote amp quote 
EMABiggestFansNickiMinaj 

RT RT Im fighting for the girls that never thought they could win EMABiggestFansNickiMinaj

RT If You See This You Betta RT EMABiggestFansNickiMinaj 

RT SO to all the ppl who said female rappers wont sell in this day and age EMABiggestFansNickiMinaj

RT The Barbz on EMABiggestFansNickiMinaj 

RT Young Onika 
EMABiggestFansNickiMinaj 
EMABiggestFansNickiMinaj 

RT ❤ RT RT If You Loved This Performance 🔥🔥 
EMABiggestFansNickiMinaj 

RT BarbzIs really gonna host the MTVEMA and NOT WIN FOR BIGGEST FANS RT  EMABiggestFansNickiMinaj http…

RT Throwback Thursday 
EMABiggestFansNickiMinaj 
EMABiggestFansNickiMinaj 

All of the barbzkenz need to be voting EMABiggestFansNickiMinaj

You love Nicki but youre not voting EMABiggestFansNickiMinaj

Nicki is hosting the EMAs and some of these barbzkenz arent voting EMABiggestFansNickiMinaj



RT My greatest regret at the passing of Americahating strongman Hugo Chavez is that he didnt live long enough to party with 

that happened to me once too The guy was ok Hope yours was too

RT Holy hell thats crazy what a sad story RT County to demolish sinkhole home on Sunday Body to remain underneath

Went to TacoBell Total came to Gave her She said You gave me a penny too much Then she gave me pennies back SMH

RT A boy and his dog  cfisher  West Dog Park 

Cutest thing EVER RT Yeah I know just another bad habit to have to break  I cont 

RT Longest magazine article ever 

RT oinks for named of Favorite Pigs on National Pig Day Snort bad for a piglet  

Time to box 

I was dripping sweat minutes in Def an ass kicking class boxing

I have to admit after boxing class I came home amp ate Oreos Bad bad girl

Boxing class was tough Much more to it than it seems Cant wait to go back

RT In teachers in Switzerland had an average salary of about per year

yep Should be fun Thanks

I start 


Why are news reports saying djs were reassigned to other jobs amp not fired Thats obv a lie Of course laid off is still fired

I havent used sucrets since elementary school but they are amazing Soothing on my sore throat Wish there was another flavor tho

at least he went

at least  is following her now

I try to show support when I can purchase books but thats ridiculous

Change of plan Next book the tragedy paper BampN selling the twelve  full  Bamphas it for less

RT On this gratituesday I am grateful for My job  too many dont have one My kids  theyre healthy amp good kids

Best book I read in  Gone Girl by Jillian Flynn Must make time to reread that one

if theyre talking about the short shorts joke my yo daughter and I loved it

she dropped the Birney years ago

There are so many classics Id love to read but my list is too long Still have a new Jen Weiner I havent gotten to

Have my next books already decided The Twelve The Tragedy Paper and Sharp Objects Hope the library gets th


I will never be able to top the gift Im getting for my oldest son this year So freakin awesome christopherwardwatch

RT Texas cop wraps bill in traffic ticket  Crimesider via 

RT At concerts when Rockstars ask the audience Are you ready to rock I always yell Wooooo even though Im not always ready

RT I dont know why people make a big deal of being once in a lifetime Every day of your life is once in a  

RT RT Former US Marine unjustly stuck in a Mexican prison freejonhammar



My daughter walked in the room amp said You are such an old lady Crocheting with the dog on your lap while watching QVC

It is deg as I crochet wool scarves for my coworkers

Crocheting in the mall parking lot while my daughter amp her BFF attend the Psy miniconcert

Dear ad agencies please stop making ads that start with In a world  Thanks

omg me too Laying in bed catching up on twitter

makes sense Hadnt heard that one

idk if its official but heard someone on tv talking about it I have an English aunt I sh

AttributeError: 'int' object has no attribute 'backward'

In [ ]:
hiddens=np.squeeze(hiddens)

In [ ]:
# hiddenDF=pd.DataFrame(hiddens)
# allFeatures = pd.concat([hiddenDF, extraFeatures], axis=1)
# allFeatures.head()

## Classifiers

In [ ]:
crisisEncoder = LabelEncoder()
y = crisisEncoder.fit_transform(data['class'])
print (crisisEncoder.classes_)
print(y)
print(data.tweet[0])

In [ ]:
log = cross_val_score(LogisticRegression(), hiddens, y,cv=2)
print("Logisitic regression average accuracy : ",log.mean())

In [ ]:
nb = cross_val_score(GaussianNB(), hiddens, y,cv=2)
print("Naive bayes average accuracy : ",nb.mean())

In [ ]:
from sklearn.ensemble import RandomForestClassifier
log = cross_val_score(RandomForestClassifier(), hiddens, y,cv=2)
print("Random forest average accuracy : ",log.mean())

In [ ]:
clf = RandomForestClassifier()
clf.fit(hiddens, y)

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
log = cross_val_score(ExtraTreesClassifier(), hiddens, y,cv=2)
print("Extra trees average accuracy : ",log.mean())

In [ ]:
logreg = LogisticRegression()
logreg.fit(hiddens, y)

## Visualize other hidden state via Logistic

In [ ]:
def evaluate():
    hidden_init = model.init_hidden(1)
    loss_avg = 0
    for s in range(1):
        hidden = hidden_init
        loss =0
        tokens = "I think the sky is blue".split(' ')
        for i,word in enumerate(tokens):
            emb = getEmb(word)
            output, hidden = model(Variable(emb), var(hidden.data))
            ## Send to Logistic
            temp=hidden.view(50)
            temp = temp.data.numpy()
            temp = temp.reshape(1,-1)
            scales=logreg.predict_proba(temp)
            label=logreg.predict(temp)
            if label==0: #control - green 
                text="\033[0;30;42m "+word
                print (text, end=" ")
            if label==1: #depressed - red
                text="\033[1;30;41m "+word
                print (text, end=" ")
            if label==2: #ptsd-blue
                text="\033[1;30;44m "+word
                print (text, end=" ")
            
            output = output.view(50,1)
            if i+1 < len(tokens):
                s1 = Variable(getEmb(tokens[i+1]).long())
                loss += criterion(output, s1.float())
        hidden_init = copy_state(hidden)
        loss_avg = .99*loss_avg + .01*loss.data[0]/len(tokens)

evaluate()

In [ ]:
#http://ozzmaker.com/add-colour-to-text-in-python/

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable

output = Variable(torch.rand(1,10))
target = Variable(torch.LongTensor([1]))

criterion = nn.CrossEntropyLoss()
loss = criterion(output, target)
print (output)
print(loss)


In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
xcm=confusion_matrix(y_test, y_pred, labels=None, sample_weight=None)
xcm=numpy.array(xcm)
ax = sns.heatmap(xcm)
plt.xlabel('Test Labels')
plt.ylabel('Predicted Labels')
#plt.show()
plt.savefig('ConfusionMatrix')